In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 2000)

from epiweeks import Week
import numpy as np
from random import randint
from math import floor, ceil
import codecs
from glob import glob

In [ ]:
from bulletin import default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label, tables
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
def macro_sort(column):
    """Sort function"""
    macros = ['Leste', 'Oeste', 'Noroeste', 'Norte', 'Fora']
    correspondence = {macro: order for order, macro in enumerate(macros)}
    return column.map(correspondence)

def evolucao_sort(column):
    evolucoes = ['Casos','Óbitos','Recuperados','Ativos']
    correspondence = {evolucao: order for order, evolucao in enumerate(evolucoes)}
    return column.map(correspondence)
    

In [ ]:
municipios = tables['municipios'].set_index('ibge')
municipios['regional'] = municipios['rs'].astype(str).str.zfill(2) + 'ª - ' + municipios['regional']
municipios = municipios.drop(['ibge7','rs'], axis=1)
municipios

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)

In [ ]:
casos_confirmados = cc.df.loc[cc.df['uf_residencia']=='PR']

In [ ]:
total_ibge = casos_confirmados.rename(columns={'ibge_residencia':'ibge'}).groupby(['ibge','evolucao','ano_caso','ano_obito'])[['identificacao']].agg({'count'}).unstack().droplevel([0,1],axis=1).fillna(0).astype(int).rename(columns={1:'Recuperados',2:'Óbitos',3:'Ativos'})
total_ibge['Casos'] = total_ibge.sum(1)
total_ibge = total_ibge

total_ibge = total_ibge.sort_index(axis=1,key=evolucao_sort)\
                       .join(municipios[['uf','macro','regional','municipio','populacao']])\
                       .reset_index()\
                       .set_index(['uf','macro','regional','ibge','municipio','populacao'])

total_ibge.index = total_ibge.index.rename({'uf':'UF','macro':'Macro','regional':'Regional','ibge':'IBGE','municipio':'Município','populacao':'População'})
total_ibge = total_ibge.reset_index('População')
total_ibge = total_ibge.sort_index(axis=0,level=1,key=macro_sort)

por_ibge = total_ibge

total_regional = total_ibge.groupby(level=[0,1,2]).sum()
total_regional[['IBGE', 'Município']] = 'Total'
total_regional.set_index(['IBGE', 'Município'], append=True, inplace=True)
total_ibge = pd.concat([total_ibge,total_regional])
total_ibge = total_ibge.sort_index(axis=0,level=1,key=macro_sort)

total_macro = total_ibge.groupby(level=[0,1]).sum()
total_macro[['Regional','IBGE', 'Município']] = 'Total'
total_macro.set_index(['Regional','IBGE', 'Município'], append=True, inplace=True)
total_ibge = pd.concat([total_ibge,total_macro])
total_ibge = total_ibge.sort_index(axis=0,level=1,key=macro_sort)

total_pr = total_ibge.groupby(level=[0]).sum()
total_pr[['Macro','Regional','IBGE', 'Município']] = 'Total'
total_pr.set_index(['Macro','Regional','IBGE', 'Município'], append=True, inplace=True)
total_ibge = pd.concat([total_ibge,total_pr])

coef = total_ibge[['Casos','Óbitos','Óbitos']] / total_ibge[['População','População','Casos']].values
coef.columns = ['Coeficiente de Incidência','Coeficiente de Mortalidade','Taxa de Letalidade']
total_ibge = total_ibge.join(coef)

In [ ]:
por_ibge.reset_index()